In [1]:
import pandas as pd
import numpy as np
import json
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import date
from datetime import datetime
from dateutil import relativedelta
import plotly.express as px


A new type of Player Card that shows some more insights

In [2]:
all_pak_matches = os.listdir('pakistan_male_json')
babar_azam_matches = []
for match in all_pak_matches:
    if match != 'README.txt':
        fname = 'pakistan_male_json/' + match
        with open(fname, 'r') as f:
            data = json.load(f)
            if 'Babar Azam' in data['info']['players']['Pakistan']:
                babar_azam_matches.append(match)

In [21]:
len(babar_azam_matches)

221

In [3]:
with open('pakistan_male_json/' + '1153250.json', 'r') as f:
        data = json.load(f)


In [4]:
data['info']['dates'][0]

'2018-09-21'

In [5]:
# this function extracts all bowls played by babar azam in a match along with other realted data of match
def get_player_balls(match,player_name):
    with open('pakistan_male_json/' + match, 'r') as f:
        data = json.load(f)
    
    all_bowls = []
    
    team1 = data['info']['teams'][0]
    team2 = data['info']['teams'][1]
    
    match_type = data['info']['match_type']
    
    toss_winner = data['info']['toss']['winner']
    toss_decision = data['info']['toss']['decision']
    match_venue = data['info']['venue']
    match_date = data['info']['dates'][0]
    
    if 'winner' in data['info']['outcome']:
        match_outcome = data['info']['outcome']['winner']
    else:
        match_outcome = data['info']['outcome']['result']

    batters = set()
    inning_num = 0
    for innings in data['innings']:
        inning_num +=1
        if innings['team'] == 'Pakistan':
            overs = innings['overs']
           
            
            for over in overs:
                for delivery in over['deliveries']:
                    batter = delivery['batter']
                    batters.add(batter)
                    if batter == player_name:
                        temp_dict = {}
                        temp_dict['match_file_name'] = match
                        temp_dict['match_type'] = match_type
                        temp_dict['toss_winner'] = toss_winner
                        temp_dict['toss_decision'] = toss_decision
                        temp_dict['match_venue'] = match_venue
                        temp_dict['match_date'] = match_date
                        temp_dict['match_against'] = team2 if team1 == 'Pakistan' else team1
                        temp_dict['outcome'] = match_outcome
                        temp_dict['inning_num'] = inning_num
                        temp_dict['match_inning'] = str(match) + '_' + str(inning_num)
                        
                        temp_dict['batter'] = batter
                        temp_dict['non_striker'] = delivery['non_striker']
                        temp_dict['runs'] = delivery['runs']['batter']
                        temp_dict['bowler'] = delivery['bowler']
                        temp_dict['extras'] = delivery['runs']['extras']

                        temp_dict['out'] = False
                        temp_dict['fielders'] = ''
                        temp_dict['wicket_kind'] = ''



                        if 'wickets' in delivery:

                            for wicket in delivery['wickets']:
                                if wicket['player_out'] == player_name:
                                    temp_dict['out'] = True
                                    wicket_kind = wicket['kind']
                                    temp_dict['wicket_kind'] = wicket_kind


                                    if 'fielders' in wicket:
                                        temp_dict['fielders'] = wicket['fielders'][0]
                                    else:
                                        fielders = ''
                        all_bowls.append(temp_dict)
#     print(batters)
    return all_bowls
        

In [6]:
a = get_player_balls('1077953.json','Babar Azam')


In [7]:
all_bowls = []
for match in babar_azam_matches: 
    balls = get_player_balls(match,'Babar Azam')
    all_bowls.extend(balls)
        

In [8]:
ba_df = pd.DataFrame(all_bowls)    

In [9]:
ba_df

,match_file_name,match_type,toss_winner,toss_decision,match_venue,match_date,match_against,outcome,inning_num,match_inning,batter,non_striker,runs,bowler,extras,out,fielders,wicket_kind
0,1077953.json,Test,Pakistan,field,"Sabina Park, Kingston",2017-04-21,West Indies,Pakistan,2,1077953.json_2,Babar Azam,Ahmed Shehzad,0,AS Joseph,0,False,,
1,1077953.json,Test,Pakistan,field,"Sabina Park, Kingston",2017-04-21,West Indies,Pakistan,2,1077953.json_2,Babar Azam,Ahmed Shehzad,0,AS Joseph,0,False,,
2,1077953.json,Test,Pakistan,field,"Sabina Park, Kingston",2017-04-21,West Indies,Pakistan,2,1077953.json_2,Babar Azam,Ahmed Shehzad,0,JO Holder,0,False,,
3,1077953.json,Test,Pakistan,field,"Sabina Park, Kingston",2017-04-21,West Indies,Pakistan,2,1077953.json_2,Babar Azam,Ahmed Shehzad,0,JO Holder,0,False,,
4,1077953.json,Test,Pakistan,field,"Sabina Park, Kingston",2017-04-21,West Indies,Pakistan,2,1077953.json_2,Babar Azam,Ahmed Shehzad,0,JO Holder,0,False,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13565,1152842.json,ODI,Pakistan,field,The Rose Bowl,2019-05-11,England,England,2,1152842.json_2,Babar Azam,Fakhar Zaman,1,CR Woakes,0,False,,
13566,1152842.json,ODI,Pakistan,field,The Rose Bowl,2019-05-11,England,England,2,1152842.json_2,Babar Azam,Asif Ali,2,AU Rashid,0,False,,
13567,1152842.json,ODI,Pakistan,field,The Rose Bowl,2019-05-11,England,England,2,1152842.json_2,Babar Azam,Asif Ali,2,AU Rashid,0,False,,
13568,1152842.json,ODI,Pakistan,field,The Rose Bowl,2019-05-11,England,England,2,1152842.json_2,Babar Azam,Asif Ali,1,AU Rashid,0,False,,


In [10]:
ba_df['match_date'] =  pd.to_datetime(ba_df['match_date'], format='%Y-%m-%d')


In [11]:
ba_df_t20 = ba_df[ba_df['match_type'] == 'T20']

In [12]:
ba_df_t20

,match_file_name,match_type,toss_winner,toss_decision,match_venue,match_date,match_against,outcome,inning_num,match_inning,batter,non_striker,runs,bowler,extras,out,fielders,wicket_kind
439,1117821.json,T20,ICC World XI,field,Gaddafi Stadium,2017-09-12,ICC World XI,Pakistan,1,1117821.json_1,Babar Azam,Ahmed Shehzad,4,M Morkel,0,False,,
440,1117821.json,T20,ICC World XI,field,Gaddafi Stadium,2017-09-12,ICC World XI,Pakistan,1,1117821.json_1,Babar Azam,Ahmed Shehzad,0,M Morkel,0,False,,
441,1117821.json,T20,ICC World XI,field,Gaddafi Stadium,2017-09-12,ICC World XI,Pakistan,1,1117821.json_1,Babar Azam,Ahmed Shehzad,4,NLTC Perera,0,False,,
442,1117821.json,T20,ICC World XI,field,Gaddafi Stadium,2017-09-12,ICC World XI,Pakistan,1,1117821.json_1,Babar Azam,Ahmed Shehzad,4,NLTC Perera,0,False,,
443,1117821.json,T20,ICC World XI,field,Gaddafi Stadium,2017-09-12,ICC World XI,Pakistan,1,1117821.json_1,Babar Azam,Ahmed Shehzad,0,NLTC Perera,0,False,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13511,1327233.json,T20,England,field,"Gaddafi Stadium, Lahore",2022-09-30,England,England,1,1327233.json_1,Babar Azam,Mohammad Nawaz,0,SM Curran,1,False,,
13512,1327233.json,T20,England,field,"Gaddafi Stadium, Lahore",2022-09-30,England,England,1,1327233.json_1,Babar Azam,Mohammad Nawaz,4,RJW Topley,0,False,,
13513,1327233.json,T20,England,field,"Gaddafi Stadium, Lahore",2022-09-30,England,England,1,1327233.json_1,Babar Azam,Mohammad Nawaz,6,RJW Topley,0,False,,
13514,1327233.json,T20,England,field,"Gaddafi Stadium, Lahore",2022-09-30,England,England,1,1327233.json_1,Babar Azam,Mohammad Nawaz,2,RJW Topley,0,False,,


In [13]:
debut = ba_df_t20.min()['match_date'].strftime('%A %d %B %Y')
print(debut)

Wednesday 07 September 2016


/var/folders/0v/j7x4s1_x5l71g3mcjv40116w0000gn/T/ipykernel_99286/26512857.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  debut = ba_df_t20.min()['match_date'].strftime('%A %d %B %Y')


In [14]:
diff = relativedelta.relativedelta(datetime.now(), ba_df_t20.min()['match_date'].to_pydatetime())

years = diff.years
months = diff.months
days = diff.days

print('{} years {} months {} days'.format(years, months, days))

6 years 1 months 11 days


/var/folders/0v/j7x4s1_x5l71g3mcjv40116w0000gn/T/ipykernel_99286/4036701574.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  diff = relativedelta.relativedelta(datetime.now(), ba_df_t20.min()['match_date'].to_pydatetime())


In [15]:
ba_df_t20['match_file_name'].nunique()

82

In [16]:
ba_df_t20['runs'].sum()

3039

In [17]:
ba_df_t20['runs'].sum()/ba_df_t20 [ba_df_t20['out'] == True].shape[0]

44.69117647058823

In [18]:
ba_df_t20 [ba_df_t20['out'] == True].shape[0]

68

In [19]:
print(ba_df_t20.groupby('match_file_name', as_index=False).first().sort_values('match_date')[['match_date','match_against','outcome','runs']].to_string())


   match_date match_against       outcome  runs
81 2016-09-07       England      Pakistan     0
0  2016-09-23   West Indies      Pakistan     0
1  2016-09-24   West Indies      Pakistan     0
2  2016-09-27   West Indies      Pakistan     0
3  2017-03-26   West Indies      Pakistan     0
4  2017-03-30   West Indies      Pakistan     4
5  2017-04-01   West Indies   West Indies     1
6  2017-04-02   West Indies      Pakistan     2
10 2017-09-12  ICC World XI      Pakistan     4
11 2017-09-13  ICC World XI  ICC World XI     1
12 2017-09-15  ICC World XI      Pakistan     1
13 2017-10-26     Sri Lanka      Pakistan     0
14 2017-10-27     Sri Lanka      Pakistan     1
15 2017-10-29     Sri Lanka      Pakistan     1
7  2018-01-22   New Zealand   New Zealand     0
8  2018-01-25   New Zealand      Pakistan     1
9  2018-01-28   New Zealand      Pakistan     2
16 2018-04-01   West Indies      Pakistan     0
17 2018-04-02   West Indies      Pakistan     0
18 2018-04-03   West Indies      Pakista

In [20]:
fig = px.bar(data_canada, x='year', y='pop')
fig.show()

NameError: name 'data_canada' is not defined

In [ ]:
ba_df_t20.groupby('match_file_name').sum()

In [ ]:
innings[(innings['runs'] > 30)  ].shape[0]

In [ ]:
innings[(innings['runs'] > 50)  ].shape[0]

In [ ]:
innings[(innings['runs'] > 100)  ].shape[0]

In [ ]:
data = dict(
    number=[innings[(innings['runs'] > 30)  ].shape[0], 
            innings[(innings['runs'] > 50)  ].shape[0], 
            innings[innings['runs'] > 100 ].shape[0]],
    stage=["30+", "50+", "100+"])
fig = px.funnel(data, x='number', y='stage')
fig.show()

In [ ]:
fig = px.funnel_area(names=["30s", "50s", "100s"],
                    values=[innings[(innings['runs'] > 30)  ].shape[0], 
            innings[(innings['runs'] > 50)  ].shape[0], 
            innings[innings['runs'] > 100 ].shape[0]])
fig.show()

# number=,
#     stage=

In [ ]:
fig = px.histogram(innings, x="runs",nbins=40)
fig.show()